<a href="https://colab.research.google.com/github/yneha70/IIITH/blob/main/Task_1_Object_detection_and_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from ultralytics import YOLO
import os
import cv2
import numpy as np

image_path = r'C:\Users\Neha\Desktop\restaurant.jpg'

if not os.path.exists(image_path):
    raise FileNotFoundError(f"Image not found at {image_path}.")

img = cv2.imread(image_path)

model = YOLO('yolov8l-seg.pt')

results = model.predict(
    source=image_path,
    device='cpu',
    conf=0.1,
    imgsz=1600,
    save=False
)

class_colors = {
    'person': (0, 0, 255),  # Red
    'cup': (0, 255, 255)    # Yellow
}

person_count = 0
cup_count = 0

for box, cls, mask in zip(results[0].boxes, results[0].boxes.cls, results[0].masks.data):
    class_id = int(cls)
    class_name = model.names[class_id].lower()

    if class_name not in ['person', 'cup']:
        continue

    confidence = float(box.conf)
    xyxy = [int(x) for x in box.xyxy.tolist()[0]]
    color = class_colors[class_name]

    if class_name == 'person':
        person_count += 1
        label = f"Person {confidence:.2f}"
    else:
        cup_count += 1
        label = f"Cup {confidence:.2f}"

    cv2.rectangle(img, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, 3)
    cv2.putText(img, label, (xyxy[0], xyxy[1]-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Overlay segmentation mask for only people and cups
    mask_np = mask.cpu().numpy().astype(bool)
    img[mask_np] = (img[mask_np] * 0.5 + np.array(color) * 0.5).astype(np.uint8)

output_path = r"C:\Users\Neha\Desktop\restaurant_people_cups.jpg"
cv2.imwrite(output_path, img)

print(f"People: {person_count}")
print(f"Cups: {cup_count}")
print(f"Output saved at {output_path}")
